# PDF info extraction

- First upload match_reports.zip (compress the original match_reports folder) and extract_pdf_standard_v1.py


## Install Packages


In [1]:
pip install pytesseract pdf2image PyPDF2 openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [2]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (532 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!apt-get install tesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package tesseract


## Unzip reports folder

In [4]:
import zipfile
import os

# Path to the zip file
zip_path = "match-reports.zip"

# Extract to this directory
extract_dir = "your_prefer_path"

# Ensure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")

Files extracted to /content/


# We focus on decoding these four main categories in the match report.

*   passsing
*   shots
*   losses
*   duels

For each category, we already determine the target keyword (ex. "PlayerTimeShot") for us to pinpoint the pages we need to extract pdf.


- **In this steps, we batch-screenshot shot useful visuals regarding the 4 categories and save it to each schools' report file.**



In [4]:
# We will import defined funcitons in the extract_pdf_standard_v1.py

# the function process_all_pdfs help extract (category) related pdfs from the match reports belongs to each school folder.

In [5]:
import os
import extract_pdf_standard_v1 as eps

def process_pdfs_in_match_reports(root_dir):
    # Iterate over each subdirectory (each school)
    for subdir in os.listdir(root_dir):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path):
            # Define input and output directories
            input_dir = subdir_path
            output_dir = f"{subdir_path}_pdf_results"
            os.makedirs(output_dir, exist_ok=True)

            eps.process_all_pdfs(input_dir, os.path.join(output_dir, "screenshot_passes"), "PASSES")
            eps.process_all_pdfs(input_dir, os.path.join(output_dir, "screenshot_shots"), "PlayerTimeShot")
            eps.process_all_pdfs(input_dir, os.path.join(output_dir, "screenshot_duels"), "DUELS")
            eps.process_all_pdfs(input_dir, os.path.join(output_dir, 'screenshot_loss'), "LOSSES")

            print(f"Processed PDFs for {subdir}")

# Usage
process_pdfs_in_match_reports("/your_prefer_path/match-reports/")


Processing: Rutgers Scarlet Knights - Maryland College Park Terrapins 3-1.pdf
Pages with keyword 'PASSES' found: [(14, '30.10.2024'), (15, '30.10.2024')]
School names extracted: ('Rutgers Scarlet Knights', 'Maryland College Park Terrapins 3-1')
Processing pages: [(14, '30.10.2024'), (15, '30.10.2024')]
Screenshot saved to /content/match-reports/maryland-2024_pdf_results/screenshot_passes/Rutgers_Scarlet_Knights_30.10.2024_top_half_PASSES.jpg
Screenshot saved to /content/match-reports/maryland-2024_pdf_results/screenshot_passes/Maryland_College_Park_Terrapins_3-1_30.10.2024_top_half_PASSES.jpg
Processing: Maryland College Park Terrapins - UCLA Bruins 0-6.pdf
Pages with keyword 'PASSES' found: [(13, '07.11.2024'), (16, '07.11.2024'), (17, '07.11.2024')]
School names extracted: ('Maryland College Park Terrapins', 'UCLA Bruins 0-6')
Processing pages: [(16, '07.11.2024'), (17, '07.11.2024')]
Screenshot saved to /content/match-reports/maryland-2024_pdf_results/screenshot_passes/Maryland_Coll

# Feed PDF to AI models

In [12]:
## 2 mins

In [13]:
import os
import base64
from openai import OpenAI
from datetime import datetime  # Ensure datetime is imported


# Initialize OpenAI client
client = OpenAI(api_key="your_api_key")


## Define dictionary for searching file path


In [ ]:
# Create the dictionary for searching file path
school_dict = {
    'rutgers': "Rutgers",
    'ohio-st': "Ohio",
    'penn-state': "Penn",
    'michigan': "Michigan_Wolverines",
    'washington': "Washington",
    'northwestern': "Northwestern",
    'mich-state': "Michigan_State",
    'maryland': "Maryland",
    'wisconsin': "Wisconsin",
    'indiana': "Indiana",
    'ucla': "UCLA"
}

## Define task/prompts for chatGPT
- Do not change the folder-name until you understand every step/function behind the extract_pdf_standard_v1

- Feel free to change the prompts for each category - the folder name indicates the category

In [ ]:
# Define the tasks
tasks = [
    {
        "folder_name": "screenshot_passes",
        "prompt": """
What are the passing strategies of the teams in these images? What are their strengths and weaknesses? Respond in less than 500 tokens.
Limit your response to 500 tokens.
"""
    },
    {
        "folder_name": "screenshot_shots",
        "prompt": """
Based on the shot patterns of {school}, tell me:
1. Where they like to kick to.
2. Where they have the highest chance of scoring.
Answer both from the position of the net, and the position on the field.
Limit your response to 500 tokens.
"""
    },
    {
        "folder_name": "screenshot_duels",
        "prompt": """
These are all the duels of {school} against other schools. Carefully look at each image and give me a summary of:
1. In which positions of the field does {school} like to duel the most.
2. Where they have the highest chance of winning.
3. Where they have the highest chance of losing.
Limit your response to 500 tokens.
"""
    },
    {
        "folder_name": "screenshot_loss",
        "prompt": """
Using the provided images, analyze the ball losses of {school} during the match. Your task is to:
1. Identify the specific areas on the field where {school} experiences the most ball losses.
2. Categorize the types of losses (e.g., forward pass, ground duel, lateral/back pass, or other) and indicate which type occurs most frequently.
3. Discuss the impact of these losses on {school}'s defensive structure and how it affects their ability to regain possession.
4. Highlight any patterns or trends, such as losses occurring in high-pressure zones, counterattack scenarios, or while attempting build-up play.
Limit your response to 500 tokens.
"""
    }
]

### ChatGPT Pipeline Logic
1. From each school's report folder, we retrieve the saved screenshot visuals for each category.
2. We feed the visuals into ChatGPT, asking predefined questions to identify useful patterns related to **Passes, Ball Losses, Duels, and Ball Shots.**
3. We save the ChatGPT responses into text files. Each school will have a text file summarizing the patterns found in the four categories.

In [17]:
# Path to the match reports and final results directory
match_reports_root = "/your_prefer_path/match-reports"
final_result_dir = "/your_prefer_path/naming_of_the_txt"
os.makedirs(final_result_dir, exist_ok=True)

# Get a list of all schools (subdirectories in match_reports_root)
schools = [
    d for d in os.listdir(match_reports_root)
    if os.path.isdir(os.path.join(match_reports_root, d)) and '_pdf_results' not in d
]

# Process each school
for school in schools:
    school_name = school_dict[school.replace('-2024', '')]
    school_input_dir = os.path.join(match_reports_root, school)
    school_output_dir = f"{school_input_dir}_pdf_results"
    os.makedirs(school_output_dir, exist_ok=True)

    # Collect responses
    responses = []

    for idx, task in enumerate(tasks):
        folder_path = os.path.join(school_output_dir, task["folder_name"])
        prompt = task["prompt"].format(school=school)
        response = process_and_query_images(client, school_name,folder_path, prompt,3)
        if response:
            task_title = [
                "Passing Strategies Analysis",
                "Shot Patterns Analysis",
                "Duels Analysis",
                "Ball Losses Analysis"
            ][idx]
            responses.append(f"{task_title}:\n{response}\n\n")

    # Write responses to file in the final_result directory
    result_file_path = os.path.join(final_result_dir, f"{school}.txt")
    with open(result_file_path, "w") as f:
        for response in responses:
            f.write(response)

    print(f"All analyses for {school} have been written to {result_file_path}")

Searching for images related to Maryland...
Searching for images related to Maryland...
Searching for images related to Maryland...
Searching for images related to Maryland...
All analyses for maryland-2024 have been written to /content/final_result_retro3/maryland-2024.txt
Searching for images related to Wisconsin...
Searching for images related to Wisconsin...
Searching for images related to Wisconsin...
Searching for images related to Wisconsin...
All analyses for wisconsin-2024 have been written to /content/final_result_retro3/wisconsin-2024.txt
Searching for images related to Indiana...
Searching for images related to Indiana...
Searching for images related to Indiana...
Searching for images related to Indiana...
All analyses for indiana-2024 have been written to /content/final_result_retro3/indiana-2024.txt
Searching for images related to Northwestern...
Searching for images related to Northwestern...
Searching for images related to Northwestern...
Searching for images related to

In [10]:

# !zip -r match_report_fill.zip /content/match-reports


  adding: content/match-reports/ (stored 0%)
  adding: content/match-reports/washington-2024_pdf_results/ (stored 0%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/ (stored 0%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/Rutgers_Scarlet_Knights_2-0_26.10.2024_top_half_PASSES.jpg (deflated 30%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/Washington_Huskies_30.08.2024_top_half_PASSES.jpg (deflated 28%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/Washington_Huskies_22.10.2024_top_half_PASSES.jpg (deflated 31%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/Wisconsin_Badgers_0-0_28.09.2024_top_half_PASSES.jpg (deflated 33%)
  adding: content/match-reports/washington-2024_pdf_results/screenshot_passes/Washington_Huskies_2-0_08.10.2024_top_half_PASSES.jpg (deflated 33%)
  adding: content/match-reports/washington-2024_pdf_results/

In [19]:
# !zip -r final_result_retro3.zip /content/final_result_retro3

updating: content/final_result_retro3/ (stored 0%)
updating: content/final_result_retro3/washington-2024.txt (deflated 61%)
updating: content/final_result_retro3/ucla-2024.txt (deflated 59%)
updating: content/final_result_retro3/maryland-2024.txt (deflated 61%)
updating: content/final_result_retro3/ohio-st-2024.txt (deflated 60%)
updating: content/final_result_retro3/indiana-2024.txt (deflated 60%)
updating: content/final_result_retro3/wisconsin-2024.txt (deflated 60%)
updating: content/final_result_retro3/mich-state-2024.txt (deflated 61%)
updating: content/final_result_retro3/northwestern-2024.txt (deflated 61%)
updating: content/final_result_retro3/michigan-2024.txt (deflated 61%)
updating: content/final_result_retro3/rutgers-2024.txt (deflated 60%)
updating: content/final_result_retro3/penn-state-2024.txt (deflated 61%)
